# Prompt Engineering

Prompt Engineering Techniques explored:

1. System message experimentation
1. Chain-of-Thought with zero-shot and few-shot prompting
1. Condense message experimentation
1. Combining all together (final prompt)
1. Also: Temperature

In [93]:
# Import libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from tree_of_thoughts.openaiModels import OpenAILanguageModel
from tree_of_thoughts.treeofthoughts import MonteCarloTreeofThoughts

import tabula

import os

from PyPDF2 import PdfReader
import pandas as df

In [94]:
# load in openai API key
os.environ['OPENAI_API_KEY'] =""

## Document used and test cases

For the purpose of experimentation, let's use the Tesla Annual Report for year 2022. This document contains both text and tabular information (at the end of document), hence it will be a good test document for our prompt engineering. The document is stored in the "./data" folder.

We should also include some basic test cases beforehand. This will make it easy to check if the prompts are giving us the correct output. Some test cases are defined here for reference.

* Numbers. To test tabular data. 

1. What are the total assets and liabilities for the year 2022 in the consolidated balance sheet? Answer: 82338 million and 36440 million.
1. What is the Income before income taxes for the year  2020, 2021 and 2022? Answer: 1,154, 6,343 and 13,719 respectively.
1. What is the Comprehensive income attributable to common stockholders for year 2022?" Answer: 12141 million


* General Questions (open-ended)
1. What does Tesla do? And what types of businesses is Tesla involved in?
1. How is the financial health of Tesla?
1. What kind of technologies does Tesla invest in?

In [95]:
# template code to be used for testing of prompts
query_1 = "What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?"
query_2 = "What is the Income before income taxes for the year  2020, 2021 and 2022?"
query_3 = "What is the Comprehensive income attributable to common stockholders for year 2022?"
queries_num = [query_1,query_2,query_3]

query_5 = "What does Tesla do? And what types of businesses is Tesla involved in?"
query_6 = "How is the financial health of Tesla?"
query_7 = "What kind of technologies does Tesla invest in?"
queries_gen = [query_5, query_6, query_7]

## General Guidelines for prompt engineering

see: https://www.promptingguide.ai/introduction/tips

* Start Simple  
Slowly add on more complexity

* Focus on the instruction  
using commands to instruct the model what you want to achieve, such as "Write", "Classify", "Summarize", "Translate", "Order", etc.  
Another recommendation is to use some clear separator like "###" to separate the instruction and context.


        ### Instruction ###  
        Translate the text below to Spanish:
        Text: "hello!"

        
* Specificity
Be very specific about the instruction and task you want the model to perform. Thinking about how specific and detailed you should be. Including too many unnecessary details is not necessarily a good approach. The details should be relevant and contribute to the task at hand.

* Avoid Impreciseness  
It's often better to be specific and direct. The analogy here is very similar to effective communication -- the more direct, the more effective the message gets across.

* To do or not to do  
Avoid saying what not to do but say what to do instead. This encourages more specificity and focuses on the details that lead to good responses from the model.



## Instruction Prompt Experimentation

Here, we shall experiment with the different kinds of system prompts. To follow the guidelines above. The instruction prompt also forms the system prompt in the ChatPromptTemplate format.



In [96]:
# try baseline case with default prompt template from RetrievalQA
FILE_PATH = "../data/Tesla_Annual_Report_2022.pdf"
loader = PyPDFLoader(FILE_PATH)
data = loader.load()
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(data, embeddings)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())

In [150]:
# Inspect what the default prompt message is.
qa.combine_documents_chain.llm_chain.prompt.messages[0].prompt

PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True)

In [98]:
# output using the default prompt message
question = query_7
qa({"query": question})

{'query': 'What kind of technologies does Tesla invest in?',
 'result': 'Tesla invests in a variety of technologies related to automotive, battery and powertrain, vehicle control and infotainment software, self-driving development and artificial intelligence, energy generation and storage, design and engineering, and financial services. Some specific areas of investment include powertrain engineering, electric powertrain systems, battery cell chemistry, vehicle control software, self-driving technologies, energy storage systems, solar energy systems, intellectual property protection, sustainable operations, and in-app upgrades for vehicles.'}

In [99]:
system_template = """### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}"""

In [100]:
print(system_template)

### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}


In [101]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

In [102]:
# output using the revised_1 prompt message
question = query_7
qa({"query": question})

{'query': 'What kind of technologies does Tesla invest in?',
 'result': "Tesla invests in a variety of technologies related to electric vehicles, energy generation, and storage. Some of the key technology areas that Tesla focuses on include:\n\n1. Powertrain Engineering: Tesla invests in the development and manufacturing of powertrain systems for electric vehicles. They design and optimize powertrain systems to be adaptable, efficient, reliable, and cost-effective.\n\n2. Battery Technology: Tesla has extensive testing and research capabilities for battery cells, packs, and systems. They have developed their own proprietary lithium-ion battery cell and improved manufacturing processes to increase energy density and reduce costs.\n\n3. Vehicle Control and Infotainment Software: Tesla develops and updates control software for vehicle performance, safety systems, charging management, and infotainment functions. They regularly release over-the-air software updates to enhance these features.

**Observations:**
* As can be seen, the new prompt gives a more detailed answer than the old prompt.
* After checking, it can be seen that both prompts are factually accurate.
* The new prompts, with more instructions and greater precision and clearly defined ## instruction ##, ## guidelines ## and ## context ## does help to give more detailed prompts from the example shown.


## Chain-of-thought with zero-shot prompting and few-shot prompting

What is chain-of-thought prompting: Introduced in Wei et al., chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

Link: https://arxiv.org/abs/2201.11903

In [103]:
# try baseline case with default prompt template from RetrievalQA
def get_text_from_pdf(fs_pdf_docs: list):
    text_output = ""
    for pdf_file in fs_pdf_docs:
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text_output = text_output + page.extract_text()
    return text_output

def get_chunk_from_text(whole_text: str):
    text_split = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " ", ""],
        chunk_size = 1000,
        chunk_overlap = 200,
        length_function = len
    )
    chunks = text_split.split_text(whole_text)
    return chunks

def get_vectorstore_from_chucks(chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts = chunks, embedding = embeddings)
    return vectorstore

text = get_text_from_pdf([FILE_PATH])
chunks = get_chunk_from_text(text)
vector_store = get_vectorstore_from_chucks(chunks)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever())

In [108]:
# output using the default prompt message
for query in queries_num:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")


What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
----------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the years 2020, 2021, and 2022 are as follows:

- 2020: $1,154 million
- 2021: $6,343 million
- 2022: $13,719 million
----------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The Comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
----------------------------------


In [106]:
# output using the default prompt message
for query in queries_gen:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")

What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is involved in several businesses. They primarily manufacture and sell electric vehicles (EVs) for both individual customers and businesses with commuting employees. They also offer charging options for their customers, including home charging solutions and partnerships with hospitality, retail, and public destinations. Additionally, Tesla markets and sells solar and energy storage products to residential, commercial, and industrial customers, as well as utilities. They provide service for their electric vehicles through company-owned service locations and Tesla Mobile Service. Tesla also engages in direct sales through their website and company-owned stores, with galleries in some locations for product education. They also have a used vehicle business that supports new vehicle sales through trade-ins.
----------------------------------
How is the financial health of Tesla?
Based on the provided context, it is

In [109]:
# Try with COT with zero-shot prompting by adding "let;s think this step by step"
system_template = "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer. Let's think step by step.\n----------------\n{context}"

In [110]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.1), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

In [111]:
# output using CoT with  zero-prompt message
for query in queries_num:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")

What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
----------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the years 2020, 2021, and 2022 are as follows:

- 2020: $1,154 million
- 2021: $6,343 million
- 2022: $13,719 million
----------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The Comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
----------------------------------


In [112]:
# output using the CoT with  zero-prompt message
for query in queries_gen:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")

What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is involved in several businesses. Firstly, they design, manufacture, and sell electric vehicles (EVs) for both individual customers and businesses with commuting employees. They also offer charging options for their customers, including home charging solutions and partnerships with hospitality, retail, and public destinations.

In addition to EVs, Tesla is also engaged in the energy generation and storage sector. They market and sell solar and energy storage products to residential, commercial, and industrial customers, as well as utilities. Their aim is to make clean energy adoption easy and cost-effective while reducing customer acquisition costs.

Tesla provides service and warranty for their electric vehicles through company-owned service locations and Tesla Mobile Service. They prioritize customer satisfaction and feedback.

Furthermore, Tesla has a direct sales model, primarily selling vehicles through 

**Observations:**
* Both the base case and the CoT zero-shot prompt gave accurate and factual results for the quantitative results. Therefore, the zero-shot prompt did not degrade the LLM output.
* The zero-shot CoT prompt gave much more detailed replies. It understands the context of the question better. The output is also more structured.

In [113]:
# Let's introduce CoT with few-shot prompting
system_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Let's think step by step.
## examples ##
Q: What is the net income from financial year 2021 and 2022 combined?
A: The net income for financial year 2021 is 300 million. The net income for financial year 2022 is 200 million. Therefore, the combined net income is 300 million plus 200 million which is 500 million.
Q: What is the gross profit?
A: Gross profit is defined as the total sales minus the cost of goods sold. Hence, I must find the total sales figure and the cost of goods sold figure. Then, I take the total sales figure minus the costs of goods sold figure.
### context ###
{context}"""

print(system_template)

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain_type_kwargs = {"prompt": chat_prompt}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.0), chain_type="stuff", retriever=vector_store.as_retriever(), chain_type_kwargs= chain_type_kwargs)

Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Let's think step by step.
## examples ##
Q: What is the net income from financial year 2021 and 2022 combined?
A: The net income for financial year 2021 is 300 million. The net income for financial year 2022 is 200 million. Therefore, the combined net income is 500 million.
Q: What is the gross profit?
A: Gross profit is defined as the total sales minus the cost of goods sold. Hence, I must find the total sales figure and the cost of goods sold figure. Then, I take the total sales figure minus the costs of goods sold figure.
### context ###
{context}


In [114]:
# output using the default prompt message
for query in queries_num:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")

What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
----------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the year 2020 is $1,154 million. The income before income taxes for the year 2021 is $6,343 million. The income before income taxes for the year 2022 is $13,719 million.
----------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
----------------------------------


In [115]:
# output using the default prompt message
for query in queries_gen:
    print(query)
    print(qa({"query": query})['result'])
    print("----------------------------------")

What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is a company that primarily focuses on the design, development, manufacturing, and sale of electric vehicles (EVs) and sustainable energy products. They are known for their innovative electric cars, such as the Model S, Model 3, Model X, and Model Y. However, Tesla is involved in various other businesses as well. 

Here are the types of businesses Tesla is involved in:

1. Electric Vehicles: Tesla is primarily known for its electric vehicles. They design, manufacture, and sell electric cars that are powered by rechargeable batteries, offering a sustainable and environmentally friendly alternative to traditional gasoline-powered vehicles.

2. Energy Generation and Storage: Tesla also markets and sells solar energy products and energy storage solutions. They offer solar panels and solar roofs for residential, commercial, and industrial customers, allowing them to generate clean energy. Additionally, Tesla provid

In [116]:
qa.combine_documents_chain.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nLet's think step by step.\n## examples ##\nQ: What is the net income from financial year 2021 and 2022 combined?\nA: The net income for financial year 2021 is 300 million. The net income for financial year 2022 is 200 million. Therefore, the combined net income is 500 million.\nQ: What is the gross profit?\nA: Gross profit is defined as the total sales minus the cost of goods sold. Hence, I must find the total sales figure and the cost of goods sold figure. Then, I take the total sales figure minus the costs of goods sold figure.\n### context ###\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables

**Observations:**
* The quantitative outputs are again, correct. The CoT with few-shot prompts did not degrade the LLM output.
* The general questions have largely the same output as the CoT with zero-shot prompting.
* Since CoT with few-shot prompts uses up much more tokens as compared to zero-shot prompting, we shall use zero-shot prompts instead.
* One thing I noticed was also that the category of questions is too broad. Hence, the prompts are not sufficient to cover most questions.

## Condense QA Prompt
The ConversationalRetrievalChain is a component of the Langchain system that combines chat history, question condensing, semantic search, and question answering to provide a response. On the backend, the ConversationalRetrievalChain performs the following steps:

1. It condenses the current question and the chat history into a standalone question. This is done to create a standalone vector for retrieval.
1. It uses a retriever, which can be created from a vector store, to look up relevant documents based on the condensed question.
1. It passes the retrieved documents and the original question to a question answering chain to generate a response.
1. It returns the answer to the user.

We can update the condense_QA_prompt that performs step 1 to summarize the current question and the chat history.

In [126]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    return_generated_question = True
)

In [127]:
# Inspect the default QA prompt
chain.question_generator.prompt

PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True)

In [128]:
# Let's use the base prompt to see how the output standalone question looks like
chat_history = []
question = query_7
print(question)
result = chain({"question": question, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_1 = "Out of all the technologies, which one is the most interesting and has the most potential?"
print(sub_question_1)
result = chain({"question": sub_question_1, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_2 = "Tell me more about the autonomous driving technology."
print(sub_question_2)
result = chain({"question": sub_question_2, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))


What kind of technologies does Tesla invest in?
Tesla invests in a variety of technologies, including autonomous driving, artificial intelligence, robotics, energy generation, energy storage, and software development. They apply their AI learnings from self-driving technology to robotics, as seen with their robotic humanoid, Optimus. They also leverage component-level technologies from their vehicles in their energy storage products and develop software for remote control and dispatch of energy storage systems. Additionally, Tesla focuses on developing proprietary technologies for Full Self-Driving (FSD), battery cells, and other advancements in the electric vehicle industry.
-------------------------------------------------
Out of all the technologies, which one is the most interesting and has the most potential?
Based on the provided context, it is difficult to determine which specific technology Tesla invests in has the most potential. However, some notable technologies mentioned in

In [129]:
# the output after condensing history and new question
result['generated_question']

'What can you tell me about the autonomous driving technology developed by Tesla?'

In [130]:
memory_template = """
Given the following chat history and a follow up question, rephrase the\
follow up question to be a standalone question.\
The follow up question may not always be based on the chat history.\
If follow up question is not based on the chat history, do not rephrase it.\
If follow up question is not based on the chat history, you should still answer it\
in the context given below.\
If the question is not related to the context below, just say that "I don't know".\
Chat History:{chat_history}\
Follow Up Question: {question}\
Standalone Question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(memory_template)

In [131]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    return_generated_question = True
)

In [132]:
# Let's use the improved prompt to see how the output standalone question looks like
chat_history = []
question = query_7
print(question)
result = chain({"question": question, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_1 = "Out of all the technologies, which one is the most interesting and has the most potential?"
print(sub_question_1)
result = chain({"question": sub_question_1, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))
print("-------------------------------------------------")

sub_question_2 = "Tell me more about the autonomous driving technology."
print(sub_question_2)
result = chain({"question": sub_question_2, "chat_history": chat_history})
print(result['answer'])
chat_history.append((question, result["answer"]))

What kind of technologies does Tesla invest in?
Tesla invests in a variety of technologies, including autonomous driving technology, artificial intelligence, robotics, energy generation, energy storage, and software development. They leverage their expertise in power electronics and battery systems from their vehicles to optimize their energy storage products. They also work on developing solar energy systems and offer charging solutions for residential, commercial, and industrial customers. Additionally, Tesla focuses on developing software capabilities for remote control and dispatch of their energy storage systems.
-------------------------------------------------
Out of all the technologies, which one is the most interesting and has the most potential?
Based on the provided context, it is difficult to determine which technology Tesla invests in has the most potential or is the most interesting. However, some notable technologies mentioned include electric vehicles, autonomous drivi

In [133]:
# the output after condensing history and new question
# for improved condensed qa prompt
result['generated_question']

"What specific advancements or features does Tesla's autonomous driving technology include?"

**Observations:**
* Using a modified condense_qa_prompt, the final input question is much better, in the sense that it is more specific and related to the context.
* The output from the new input question is also mush more elaborate, detailed and specific.

## Combining all together (final prompt template used)
Now that we have experimented with different methods of prompt engineering and observed the outputs, we can combine the knowledge to generate the eventual prompt used for the app deployment.

**Main observations from prompt engineering:**
1. Modifying the system prompt with instructions, guidelines and context gives answers with more details and nuances. The answers are factual and accurate.
1. CoT with zero-shot prompting is able to produce answers which are relevant and contextual. It is also highly verbose and explains with rich details on how it arrived at the final answer.
1. CoT with few-shot prompting gave similar answers to CoT with zero-shot prompting. This method also used more tokens, leaving less token space available for the context. Hence, it is better not to use CoT with few-shot prompting for our use case.
1. Using a modified condense_qa_prompt generates better subsequent questions and therefore a more specific and elaborate final output answer.

In [134]:
# QA prompt
system_template = """### Instruction ###
Use the following pieces of context to answer the users question.
### Guidelines ###
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Your role is as a financial analyst
No matter what the question is, you should always answer it in the context provided below.
If you are unsure of the answer, just say "I do not know"
### context ###
{context}"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain_type_kwargs = {"prompt": chat_prompt}


In [135]:
# Condensed QA prompt
memory_template = """
Given the following chat history and a follow up question, rephrase the\
follow up question to be a standalone question.\
The follow up question may not always be based on the chat history.\
If follow up question is not based on the chat history, do not rephrase it.\
If follow up question is not based on the chat history, you should still answer it\
in the context given below.\
If the question is not related to the context below, just say that "I don't know".\
Chat History:{chat_history}\
Follow Up Question: {question}\
Standalone Question:
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(memory_template)

In [136]:
# Final Chain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    output_key="answer",
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs= chain_type_kwargs
)


Now that we have created our final prompt template, let's run through all the 7 queries and see the output of them.

In [137]:
for query in queries_num:
    chat_history = []
    result = chain({"question": query, "chat_history": chat_history})
    print(result['question'])
    print(result['answer'])
    print("---------------------------------------------")

What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
---------------------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the year 2020 was $1,154 million, for the year 2021 was $6,343 million, and for the year 2022 was $13,719 million.
---------------------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The Comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
---------------------------------------------


In [138]:
for query in queries_gen:
    chat_history = []
    result = chain({"question": query, "chat_history": chat_history})
    print(result['question'])
    print(result['answer'])
    print("---------------------------------------------")

What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is a company that primarily focuses on the design, development, manufacturing, and sale of electric vehicles. They also provide energy generation and storage solutions through their solar and energy storage products. Additionally, Tesla offers services such as vehicle maintenance and repairs.

In terms of the types of businesses Tesla is involved in, they work with a wide range of industries. They collaborate with hospitality, retail, and public destinations to provide charging options for electric vehicles. They also work with businesses that have commuting employees to offer charging solutions. Furthermore, Tesla markets and sells their solar and energy storage products to residential, commercial, and industrial customers, as well as utilities.
---------------------------------------------
How is the financial health of Tesla?
As a financial analyst, I can provide some insights into Tesla's financial health 

## Also: Temperature
While not really a prompt engineering technique, changing the value of the temperature can give us more factual answers.

 The LLM temperature is a hyperparameter that regulates the randomness, or creativity, of the AI’s responses. A higher temperature value typically makes the output more diverse and creative but might also increase its likelihood of straying from the context. Conversely, a lower temperature value makes the AI’s responses more focused and deterministic, sticking closely to the most likely prediction.

 Setting the temperature to be lower will give more accurate response, since the sampling of the probabilities only pick those with highest probabilities.

In [141]:
# List of temperatures to test
tem_list = [0.1,0.5,1]
for tem in tem_list:
    
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=tem)

    qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = tem), chain_type="stuff", retriever=vector_store.as_retriever())

    print(f"Temperature of LLM is set to: {tem}")
    for query in queries_num:
        print(query)
        print(qa({"query": query})['result'])
        print("----------------------------------")

Temperature of LLM is set to: 0.1
What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the consolidated balance sheet are $82,338 million. The total liabilities for the year 2022 in the consolidated balance sheet are $36,440 million.
----------------------------------
What is the Income before income taxes for the year  2020, 2021 and 2022?
The income before income taxes for the years 2020, 2021, and 2022 are as follows:

- 2020: $1,154 million
- 2021: $6,343 million
- 2022: $13,719 million
----------------------------------
What is the Comprehensive income attributable to common stockholders for year 2022?
The Comprehensive income attributable to common stockholders for the year 2022 is $12,141 million.
----------------------------------
Temperature of LLM is set to: 0.5
What are the total assets and liabilities for the year 2022 in the consolidated balance sheet?
The total assets for the year 2022 in the 

In [143]:
# List of temperatures to test
tem_list = [0.1,0.5,1]
for tem in tem_list:
    
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=tem)

    qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = tem), chain_type="stuff", retriever=vector_store.as_retriever())

    print(f"Temperature of LLM is set to: {tem}")
    # output using the default prompt message
    for query in queries_gen:
        print(query)
        print(qa({"query": query})['result'])
        print("----------------------------------")

Temperature of LLM is set to: 0.1
What does Tesla do? And what types of businesses is Tesla involved in?
Tesla is involved in several businesses. They primarily manufacture and sell electric vehicles (EVs) for both individual customers and businesses with commuting employees. They also offer charging options for their customers, including home charging solutions and partnerships with hospitality, retail, and public destinations. Additionally, Tesla markets and sells solar and energy storage products to residential, commercial, and industrial customers, as well as utilities. They provide service for their electric vehicles through company-owned service locations and Tesla Mobile Service. Tesla also engages in direct sales through their website and company-owned stores, with galleries in some locations for product education. They also have a used vehicle business that supports new vehicle sales by integrating trade-ins.
----------------------------------
How is the financial health of Te

**Observations:**
* All values of the temperature from 0.1,0.5 and 1 gave accurate answers for the numerical questions. This could be because the context used was highly relevant.
* In therms of the general questions, all temperature values gave outputs that are factual. No hallucination seen.This could be because the context used was highly relevant.
* The higher temperature output gives responses that are more interesting and diversed